# Entraînement du Modèle Morningstar sur Google Colab

Ce notebook guide à travers les étapes nécessaires pour entraîner le modèle hybride multi-tâches Morningstar dans l'environnement Google Colab.

**Pré-requis :** Assurez-vous que votre projet local (contenant ce notebook et le reste du code) est synchronisé sur votre Google Drive, par exemple dans `MyDrive/CryptoRobot/Morningstar`.

## 1. Configuration de l'Environnement

### 1.1 Monter Google Drive et Définir le Chemin du Projet

In [ ]:
from google.colab import drive
from pathlib import Path
import os
import sys

drive.mount('/content/drive')

# --- IMPORTANT : Adaptez ce chemin si nécessaire --- 
PROJECT_ROOT = Path('/content/drive/MyDrive/CryptoRobot/Morningstar')

if not PROJECT_ROOT.exists():
    raise FileNotFoundError(f"Le répertoire du projet n'a pas été trouvé à : {PROJECT_ROOT}. Vérifiez le chemin.")

# Se déplacer dans le répertoire du projet
os.chdir(PROJECT_ROOT)
print(f"Répertoire courant : {os.getcwd()}")

# Ajouter le répertoire racine au PYTHONPATH pour les imports locaux
if str(PROJECT_ROOT) not in sys.path:
    print(f"Ajout de {PROJECT_ROOT} au sys.path")
    sys.path.append(str(PROJECT_ROOT))

### 1.2 Installation des Dépendances et Vérification GPU

In [ ]:
print("--- Installation des dépendances ---")
%pip install -r requirements.txt

import tensorflow as tf
print("\n--- Vérification GPU ---")
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU disponible : {gpu_devices}")
else:
    print("Aucun GPU détecté. L'entraînement sera sur CPU (plus lent).")

### 1.3 Exécution du Pipeline de Données (Optionnel)

Si les données traitées (`data/processed/*.parquet`) ne sont pas déjà sur votre Drive, vous pouvez exécuter le pipeline ici. Sinon, cette étape peut être sautée.

In [ ]:
# Décommentez pour exécuter le pipeline si nécessaire
# print("\n--- Lancement du pipeline complet de données ---")
# !python tests/manual_tests/test_full_pipeline_all_assets.py
# print("\n--- Pipeline de données terminé ---")
# !ls -l data/processed

## 2. Chargement et Préparation des Données

In [ ]:
import pandas as pd
import numpy as np

# Importer les modules locaux (le PYTHONPATH a été configuré précédemment)
from model.training.data_loader import load_and_split_data
from model.architecture.morningstar_model import MorningstarModel

# --- Configuration --- 
ASSET_NAME = 'sol' # Choisir l'actif (btc, eth, sol, etc.)
DATA_DIR = Path('data/processed') # Chemin relatif à PROJECT_ROOT
FILE_PATH = DATA_DIR / f"{ASSET_NAME}_final.parquet"
# --- IMPORTANT : Ces labels doivent correspondre aux sorties du modèle --- 
LABEL_COLUMNS = ['signal', 'volatility_quantiles', 'volatility_regime', 'market_regime', 'sl_tp']
VALIDATION_SPLIT = 0.2 # % des données pour la validation (fin de série)

print(f"Chargement des données pour : {ASSET_NAME} depuis {FILE_PATH}")

# Charger les données en tant que Tensors
X_technical = None
X_llm = None
y_dict = None
if FILE_PATH.exists():
    try:
        (X_technical, X_llm), y_dict = load_and_split_data(
            FILE_PATH, 
            label_columns=LABEL_COLUMNS, 
            as_tensor=True
        )
        print(f"Données chargées : X_technical shape={X_technical.shape}, X_llm shape={X_llm.shape}, Labels={list(y_dict.keys())}")
    except ValueError as e:
        print(f"ERREUR lors du chargement/split : {e}")
        print("Vérifiez que les LABEL_COLUMNS ci-dessus correspondent aux colonnes dans le fichier Parquet et que les embeddings LLM (simulés ou réels) sont présents.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue lors du chargement : {e}")
else:
     print(f"ERREUR : Le fichier {FILE_PATH} n'a pas été trouvé. Vérifiez le chemin et/ou exécutez le pipeline de données.")

# Continuer seulement si les données ont été chargées correctement
if X_technical is not None and X_llm is not None and y_dict is not None:
    # Séparation Train/Validation (temporelle)
    num_samples = X_technical.shape[0]
    num_val_samples = int(num_samples * VALIDATION_SPLIT)
    num_train_samples = num_samples - num_val_samples

    # Split pour les features
    X_technical_train = X_technical[:num_train_samples]
    X_technical_val = X_technical[num_train_samples:]
    
    X_llm_train = X_llm[:num_train_samples]
    X_llm_val = X_llm[num_train_samples:]

    # Split pour les labels
    y_train = {name: tensor[:num_train_samples] for name, tensor in y_dict.items()}
    y_val = {name: tensor[num_train_samples:] for name, tensor in y_dict.items()}

    # Préparation des dictionnaires d'inputs pour le modèle
    X_train = {'technical_input': X_technical_train, 'llm_input': X_llm_train}
    X_val = {'technical_input': X_technical_val, 'llm_input': X_llm_val}

    print(f"Séparation Train/Validation : Train={num_train_samples}, Val={num_val_samples}")
    print(f"Shapes : X_technical_train={X_technical_train.shape}, X_llm_train={X_llm_train.shape}")
    print(f"Labels Train : {[f'{k}:{v.shape}' for k, v in y_train.items()]}")
else:
    print("\nArrêt prématuré car les données n'ont pas pu être chargées correctement.")
    # Assigner des valeurs vides pour éviter les erreurs dans les cellules suivantes si on les exécute quand même
    X_train, X_val, y_train, y_val = None, None, {}, {}

## 3. Initialisation et Compilation du Modèle

In [ ]:
if X_train is not None:
    model_wrapper = MorningstarModel()
    model_wrapper.initialize_model()
    model = model_wrapper.model # Accéder au modèle Keras sous-jacent
    
    # --- IMPORTANT : Ces clés doivent correspondre aux noms des sorties du modèle --- 
    losses = {
        'signal': 'sparse_categorical_crossentropy',
        'volatility_quantiles': 'mse',
        'volatility_regime': 'sparse_categorical_crossentropy',
        'market_regime': 'sparse_categorical_crossentropy',
        'sl_tp': 'mse'
    }
    
    metrics = {
        'signal': ['accuracy'],
        'volatility_quantiles': ['mae'],
        'volatility_regime': ['accuracy'],
        'market_regime': ['accuracy'],
        'sl_tp': ['mae']
    }
    
    model.compile(
        optimizer='adam',
        loss=losses,
        metrics=metrics
    )
    
    print("Modèle initialisé et compilé avec succès.")
    print(model_wrapper.get_model_summary()) # Afficher le résumé via le wrapper
else:
    print("Définition/Compilation du modèle sautée car les données n'ont pas été chargées.")
    model = None

## 4. Entraînement du Modèle

In [ ]:
if model is not None and X_train is not None:
    # --- Configuration de l'Entraînement ---
    EPOCHS = 50
    BATCH_SIZE = 32
    # Le modèle sera sauvegardé dans le répertoire du projet sur Drive
    MODEL_SAVE_DIR = Path('model/training') 
    MODEL_SAVE_PATH = MODEL_SAVE_DIR / f'{ASSET_NAME}_morningstar_colab.h5'

    # Créer le répertoire de sauvegarde si nécessaire
    MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True)

    # Callbacks
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=MODEL_SAVE_PATH,
        save_weights_only=False,
        monitor='val_loss', # Surveiller la perte totale de validation
        mode='min',
        save_best_only=True,
        verbose=1
    )
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=10, # Nb epochs sans amélioration avant arrêt
        verbose=1,
        restore_best_weights=True
    )

    print(f"Début de l'entraînement pour {EPOCHS} epochs...")
    print(f"Le meilleur modèle sera sauvegardé dans : {MODEL_SAVE_PATH}")

    history = model.fit(
        X_train, # Dictionnaire {'technical_input': ..., 'llm_input': ...}
        y_train, # Dictionnaire de labels
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint_callback, early_stopping_callback],
        verbose=1
    )

    print("Entraînement terminé.")
else:
    print("Entraînement sauté car les données ou le modèle ne sont pas prêts.")
    history = None # Pour éviter les erreurs suivantes

## 5. Évaluation et Visualisation

In [ ]:
if model is not None and X_val is not None:
    print("\nÉvaluation du meilleur modèle sur l'ensemble de validation...")
    results = model.evaluate(X_val, y_val, batch_size=BATCH_SIZE, verbose=0)

    print("Résultats de l'évaluation:")
    results_dict = {}
    try:
        for name, value in zip(model.metrics_names, results):
            results_dict[name] = value
            print(f"  - {name}: {value:.4f}")
    except AttributeError:
        print("Impossible de récupérer model.metrics_names, affichage brut:", results)
    
    # Visualisation des courbes d'apprentissage
    if history is not None:
        import matplotlib.pyplot as plt
        history_dict = history.history
        print("\nClés disponibles dans l'historique:", history_dict.keys())

        # Créer les graphiques
        if 'loss' in history_dict and 'val_loss' in history_dict:
            epochs_range = range(1, len(history_dict['loss']) + 1)
            plt.figure(figsize=(15, 10))

            # Perte Totale
            plt.subplot(2, 3, 1)
            plt.plot(epochs_range, history_dict['loss'], label='Train Loss')
            plt.plot(epochs_range, history_dict['val_loss'], label='Validation Loss')
            plt.title('Total Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()

            # Métriques spécifiques (vérifier les noms exacts dans history_dict.keys())
            metric_keys = [
                ('signal_accuracy', 'Signal Accuracy'),
                ('volatility_quantiles_mae', 'Volatility Quantiles MAE'),
                ('volatility_regime_accuracy', 'Volatility Regime Accuracy'),
                ('market_regime_accuracy', 'Market Regime Accuracy'),
                ('sl_tp_mae', 'SL/TP MAE')
            ]
            
            for i, (key_base, title) in enumerate(metric_keys, start=2):
                train_key = key_base
                val_key = f'val_{key_base}'
                if train_key in history_dict and val_key in history_dict:
                    plt.subplot(2, 3, i)
                    plt.plot(epochs_range, history_dict[train_key], label=f'Train {title}')
                    plt.plot(epochs_range, history_dict[val_key], label=f'Validation {title}')
                    plt.title(title)
                    plt.xlabel('Epoch')
                    plt.ylabel(title.split()[-1]) # Utilise le dernier mot comme label Y
                    plt.legend()
                else:
                    print(f"Métriques '{train_key}' ou '{val_key}' non trouvées dans l'historique.")

            plt.tight_layout()
            plt.show()
        else:
            print("Données de perte non trouvées dans l'historique.")
else:
    print("\nÉvaluation et visualisation sautées.")

## Fin du Notebook